In [ ]:
from flask import Flask, request, render_template, jsonify, url_for, flash, redirect
import os
import uuid
from werkzeug.utils import secure_filename
import base64
from io import BytesIO
import matplotlib.pyplot as plt
import seaborn as sns

app = Flask(__name__)
app.config.from_object(Config)

# Initialize components
config = Config()
config.create_directories()
image_processor = ImageProcessor(target_size=config.IMAGE_SIZE)
model_trainer = ModelTrainer(config)

# Load pre-trained model (you can train your own or use a demo model)
try:
    # Try to load existing model
    model_path = config.MODEL_FOLDER / f'{config.MODEL_NAME}.h5'
    if model_path.exists():
        model_trainer.load_model(str(model_path))
        logger.info("Model loaded successfully")
    else:
        # Create a demo model with common classes
        demo_classes = ['cat', 'dog', 'bird', 'car', 'flower', 'person', 'building', 'food', 'nature', 'object']
        model_trainer.create_model(len(demo_classes), 'mobilenet')
        model_trainer.class_names = demo_classes
        logger.info("Demo model created")
except Exception as e:
    logger.error(f"Error loading model: {str(e)}")

def allowed_file(filename):
    """Check if file extension is allowed"""
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in config.ALLOWED_EXTENSIONS

@app.route('/')
def index():
    """Main page"""
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    """Handle file upload and prediction"""
    try:
        if 'file' not in request.files:
            return jsonify({'error': 'No file provided'}), 400
        
        file = request.files['file']
        if file.filename == '':
            return jsonify({'error': 'No file selected'}), 400
        
        if file and allowed_file(file.filename):
            # Save uploaded file
            filename = secure_filename(file.filename)
            unique_filename = f"{uuid.uuid4()}_{filename}"
            file_path = config.UPLOAD_FOLDER / unique_filename
            file.save(file_path)
            
            # Process image
            try:
                # Preprocess for prediction
                processed_image = image_processor.preprocess_image(str(file_path))
                
                # Extract features
                features = image_processor.extract_features(str(file_path))
                
                # Make prediction
                predictions = model_trainer.predict(processed_image)
                
                # Enhance image (optional)
                enhanced_path = image_processor.enhance_image(str(file_path))
                
                # Prepare response
                response = {
                    'success': True,
                    'filename': unique_filename,
                    'predictions': predictions,
                    'features': features,
                    'image_url': url_for('static', filename=f'uploads/{unique_filename}')
                }
                
                return jsonify(response)
                
            except Exception as e:
                logger.error(f"Error processing image: {str(e)}")
                return jsonify({'error': f'Error processing image: {str(e)}'}), 500
        
        else:
            return jsonify({'error': 'File type not allowed'}), 400
            
    except Exception as e:
        logger.error(f"Error in upload: {str(e)}")
        return jsonify({'error': f'Upload failed: {str(e)}'}), 500

@app.route('/predict_url', methods=['POST'])
def predict_from_url():
    """Handle prediction from image URL"""
    try:
        data = request.get_json()
        image_url = data.get('url')
        
        if not image_url:
            return jsonify({'error': 'No URL provided'}), 400
        
        # Download image (simplified - in production, add proper validation)
        import requests
        response = requests.get(image_url)
        
        if response.status_code == 200:
            # Save temporary file
            unique_filename = f"{uuid.uuid4()}_url_image.jpg"
            file_path = config.UPLOAD_FOLDER / unique_filename
            
            with open(file_path, 'wb') as f:
                f.write(response.content)
            
            # Process image
            processed_image = image_processor.preprocess_image(str(file_path))
            features = image_processor.extract_features(str(file_path))
            predictions = model_trainer.predict(processed_image)
            
            response_data = {
                'success': True,
                'predictions': predictions,
                'features': features,
                'image_url': url_for('static', filename=f'uploads/{unique_filename}')
            }
            
            return jsonify(response_data)
        
        else:
            return jsonify({'error': 'Failed to download image'}), 400
            
    except Exception as e:
        logger.error(f"Error in URL prediction: {str(e)}")
        return jsonify({'error': f'URL prediction failed: {str(e)}'}), 500

@app.route('/model_info')
def model_info():
    """Get model information"""
    try:
        info = {
            'model_name': config.MODEL_NAME,
            'classes': model_trainer.class_names,
            'num_classes': len(model_trainer.class_names),
            'image_size': config.IMAGE_SIZE,
            'confidence_threshold': config.CONFIDENCE_THRESHOLD
        }
        return jsonify(info)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/batch_predict', methods=['POST'])
def batch_predict():
    """Handle batch prediction for multiple images"""
    try:
        files = request.files.getlist('files')
        if not files:
            return jsonify({'error': 'No files provided'}), 400
        
        results = []
        
        for file in files:
            if file and allowed_file(file.filename):
                # Save file
                filename = secure_filename(file.filename)
                unique_filename = f"{uuid.uuid4()}_{filename}"
                file_path = config.UPLOAD_FOLDER / unique_filename
                file.save(file_path)
                
                # Process and predict
                try:
                    processed_image = image_processor.preprocess_image(str(file_path))
                    predictions = model_trainer.predict(processed_image)
                    
                    results.append({
                        'filename': filename,
                        'predictions': predictions,
                        'image_url': url_for('static', filename=f'uploads/{unique_filename}')
                    })
                except Exception as e:
                    results.append({
                        'filename': filename,
                        'error': str(e)
                    })
        
        return jsonify({'results': results})
        
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.errorhandler(413)
def too_large(e):
    """Handle file too large error"""
    return jsonify({'error': 'File too large'}), 413

@app.errorhandler(500)
def internal_error(e):
    """Handle internal server error"""
    return jsonify({'error': 'Internal server error'}), 500

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)